In [1]:
import numpy as np
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import *
import qiskit

In [14]:
num_qubits = 64

In [15]:
def create_random_states():

    alice_state = np.random.randint(0, 2, size=num_qubits)
    bob_basis = np.random.randint(0, 2, size=num_qubits)
#     print(f"Alice's State:\t {np.array2string(alice_state, separator='')}")
#     print(f"Bob's Basis:\t {np.array2string(bob_basis, separator='')}")
    
    return alice_state, bob_basis

In [16]:
# function to define the circuit for an exchange of bit string
def make_b92_circ(enc_state, meas_basis):
    
    num_qubits = len(enc_state)

    b92 = QuantumCircuit(num_qubits)

    for index in range(len(enc_state)):
      if enc_state[index] == 1:
        b92.h(index)

    
    b92.barrier()  

    for index in range(len(meas_basis)):
      if meas_basis[index] == 1:
        b92.h(index)

    # Do not change below this line
    b92.measure_all()
    return b92

In [17]:
def b92_sifting(enc_state, meas_basis, meas_result):
    
    num_qubits = len(enc_state)
    
    sender_key = ''
    receiver_key = ''
    
    for i in range(len(meas_result)):
      if meas_result[i] == '1':
        sender_key+=str(enc_state[i])
        receiver_key+=str((meas_basis[i]+1)%2)
        
    return (sender_key, receiver_key)

In [18]:
key_list = []
iterations = 100

for index in range (0,iterations):

    alice_state, bob_basis = create_random_states()

    #Alice sends the bits to Bob but Eve eavesdrops in between
    b92_circ = make_b92_circ(alice_state, bob_basis)

    #final result got by Eve after measurement
    bob_res = execute(
            b92_circ.reverse_bits(),
            backend=QasmSimulator(),
            shots=1, 
            seed_simulator=10
        ).result().get_counts().most_frequent()


    # key sifting takes place to get the final key to be used
    alice_key, bob_key = b92_sifting(alice_state, bob_basis, bob_res)
    
#     print(f"Alice's final key: \t{alice_key}")
#     print(f"Bob's final key: \t{bob_key}")
    key_list.append(bob_key)
#     print(f"Final key: {bob_key} ")
    
avg_key_length = sum( map( len, key_list))/ len( key_list)
print(f"Initial key length = {num_qubits}")
print(f"Final average key length after {iterations} iterations =  {avg_key_length}")

Initial key length = 64
Final average key length after 100 iterations =  13.97
